# Utilisation de l'API Youtube pour rechercher des vidéos

# 0 Creation d'un projet sur Google Cloud Console

**Pour utiliser, la YouTube Data API v3 qui nous servira par la suite, il est nécessaire de créer un projet sur Google Cloud Console.**

Etape 1 : Créer un projet sur Google Cloud Console
-Aller sur [la Google Cloud Console](https://console.cloud.google.com).
-Créer un nouveau projet en cliquant sur "Select a project" puis "New Project".
-Donner un nom à votre projet et cliquer sur "Create".

Etape 2 : Activer l'API YouTube Data
-Aller dans le "API & Services" > "Library" dans la Google Cloud Console.
-Chercher YouTube Data API v3 et cliquez dessus.
-Cliquer sur "Enable" pour activer l'API.

Etape 3 : Créer des identifiants pour l'API
- Aller dans "API & Services" > "Credentials".
- Cliquer sur "Create Credentials" et choisir "API key".
- L'API key générée apparaîtra. Copier cette clé, vous en aurez besoin pour effectuer des requêtes.

## 1 Recherche de vidéos Youtube concernant des livres publiés en 2023

In [ ]:
import requests

# Remplacer key par sa propre clé API

# Fonction python pour rechercher des vidéos sur YouTube concernant des livres publiés en 2023
def get_videos(query, key):
    url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&q={query}&type=video&key={key}'
    # part=snippet : récupération des informations de base sur la vidéo (titre, description, etc.)  
    # q=nouveauté+livre+2023 : recherche avec des mots-clés  
    # type=video : récupération uniquement de vidéos (pas de playlists ou de chaînes)  
    # key= Renseignement de la clé API  
    response = requests.get(url) # Requête recherchant des vidéos sur YouTube concernant des livres publiés en 2023
    return response.json()

# La réponse de l'API contient des informations sur les vidéos comme :  
# videoId : L'ID de la vidéo qui peut être utilisé pour récupérer les sous-titres et d'autres informations  
# title : Le titre de la vidéo  
# description : La description de la vidéo  

# Fonction python pour lister les sous-titres
def get_captions(video_id, key):
    url = f'https://www.googleapis.com/youtube/v3/captions?videoId={video_id}&key={key}'
    response = requests.get(url) # Requête listant les sous-titres
    return response.json()

# Si des sous-titres existent pour une vidéo,on peut ensuite récupérer le contenu en utilisant l'ID du sous-titre.

# Fonction python pour récupérer le texte des sous-titres
def get_captionstext(caption_id, key):
    url = f'https://www.googleapis.com/youtube/v3/captions/{caption_id}?key={key}'
    response = requests.get(url) # Requête listant les textes des sous-titres
    return response.json()

# La requête n'est pas utilisable, on obtient un message d'erreur:
# 'code': 401, 'message': 'API keys are not supported by this API. Expected OAuth2 access token or other authentication credentials that assert a principal.
# Il nous manque une autorisation

# Recherche des vidéos sur YouTube concernant des livres publiés en 2023
query = "nouveauté livre 2023"
videos = get_videos(query, key)

for video in videos['items']: # Pour chaque vidéo obtenue grâce à la requête,
    video_id = video['id']['videoId'] # on récupère l'identifiant de la vidéo
    title = video['snippet']['title'] # on récupère le titre de la vidéo
    description = video['snippet']['description'] # on récupère la description de la vidéo
    print(f"Titre: {title}\nDescription: {description}\n") # on renvoie le titre et la description

    # Récupérer les sous-titres si disponibles
    # captions = get_captions(video_id, key)
    # if 'items' in captions: # S'il y a des sous-titres à la vidéo,
        # for caption in captions['items']:
            # caption_id = caption['id'] # on récupère l'identifiant des sous-titres
            # captiontext = get_captionstext(caption_id, key)
            # print(f"Sous-titres disponibles: {captiontext}\n") # on renvoie l'identifiant
    # else: # Sinon
        # print("Pas de sous-titres disponibles.") # on renvoie le message : "Pas de sous-titres disponibles"


Essayons d'obtenir aussi les sous-titres des vidéos

Vérifions que l'on se situe au bon endroit

In [ ]:
import os
print("Current working directory:", os.getcwd())

In [ ]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
import os
import pickle
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import requests

# Scopes nécessaires pour l'API YouTube
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']

# Remplacer client_secret.json par son propre fichier OAuth

# Fonction pour l'authentification OAuth2
def authenticate_youtube(client_secret.json):
    credentials = None
    # Le fichier token.pickle stocke le jeton d'accès OAuth2
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)

    # Si aucune crédentiale n'est disponible ou si elles sont invalides
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '/home/onyxia/work/pythonds_MM_MM_DS/{client_secret.json}', SCOPES)
            credentials = flow.run_local_server(port=0)

        # Sauvegarder le jeton pour les futures utilisations
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    # Retourner l'objet authentifié pour interagir avec l'API YouTube
    youtube = build('youtube', 'v3', credentials=credentials)
    return youtube

# Fonction pour rechercher des vidéos YouTube
def get_videos(query, youtube):
    request = youtube.search().list(part="snippet", q=query, type="video")
    response = request.execute()
    return response

# Fonction pour lister les sous-titres disponibles
def get_captions(youtube, video_id):
    request = youtube.captions().list(part="snippet", videoId=video_id)
    response = request.execute()
    return response

# Fonction pour récupérer les sous-titres en texte
def get_captionstext(youtube, caption_id):
    request = youtube.captions().download(id=caption_id)
    response = request.execute()
    return response

# Authentifier l'utilisateur avec OAuth2
youtube = authenticate_youtube()

# Recherche des vidéos sur YouTube concernant des livres publiés en 2023
query = "nouveauté livre 2023"
videos = get_videos(query, youtube)

for video in videos['items']:
    video_id = video['id']['videoId']
    title = video['snippet']['title']
    description = video['snippet']['description']
    print(f"Titre: {title}\nDescription: {description}")

    # Récupérer les sous-titres si disponibles
    captions = get_captions(youtube, video_id)
    if 'items' in captions:
        for caption in captions['items']:
            caption_id = caption['id']
            caption_text = get_captionstext(youtube, caption_id)
            print(f"Sous-titres disponibles: {caption_text}\n")
    else:
        print("Pas de sous-titres disponibles.")

# Cela n'a pas aboutit, après s'être connecté au compte gmail propriétaire du projet, on tombe sur ce message d'erreur

# La connexion a échoué

# Firefox ne peut établir de connexion avec le serveur à l’adresse localhost:47283.

    # Le site est peut-être temporairement indisponible ou surchargé. Réessayez plus tard ;
    # Si vous n’arrivez à naviguer sur aucun site, vérifiez la connexion au réseau de votre ordinateur ;
    # Si votre ordinateur ou votre réseau est protégé par un pare-feu ou un proxy, assurez-vous que Firefox est autorisé à accéder au Web.


Essayons d'obtenir plus de résultats

In [ ]:
import pandas as pd
import requests

# Remplacez key par sa propre clé API

# Fonction pour récupérer les vidéos
def get_videos(query, max_results=50, key):
    url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&q={query}&type=video&maxResults={max_results}&key={key}'
    response = requests.get(url)  # Requête pour rechercher des vidéos
    data = response.json()

    videos = []
    for video in data['items']:
        video_info = {
            'video_id': video['id']['videoId'],
            'title': video['snippet']['title'],
            'description': video['snippet']['description'],
            'published_at': video['snippet']['publishedAt'],
            'channel_title': video['snippet']['channelTitle']
        }
        videos.append(video_info)

    # Récupérer les pages suivantes si nécessaire
    next_page_token = data.get('nextPageToken')
    while next_page_token:
        url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&q={query}&type=video&maxResults={max_results}&pageToken={next_page_token}&key={key}'
        response = requests.get(url)
        data = response.json()
        for video in data['items']:
            video_info = {
                'video_id': video['id']['videoId'],
                'title': video['snippet']['title'],
                'description': video['snippet']['description'],
                'published_at': video['snippet']['publishedAt'],
                'channel_title': video['snippet']['channelTitle']
            }
            videos.append(video_info)
        next_page_token = data.get('nextPageToken')

    return videos

# Appel de la fonction pour récupérer les vidéos
query = "nouveauté livre 2023"
videos = get_videos(query, max_results=50, key)

# Convertir les données en DataFrame
df = pd.DataFrame(videos)

# Sauvegarder le DataFrame sous forme de fichier CSV
df.to_csv('youtube_nouveauté_livre_2023.csv', index=False)

print("Le fichier CSV a été créé avec succès !")

Le fichier CSV a été créé avec succès !


On veut se limiter aux vidéos sorties entre 2019 et 2023

In [ ]:
import pandas as pd
import requests

# Remplacez key par sa propre clé API

# Fonction pour récupérer les vidéos avec des dates spécifiques
def get_videos(query, start_date="2019-01-01T00:00:00Z", end_date="2023-12-31T23:59:59Z", max_results=50,key):
    url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&q={query}&type=video&maxResults={max_results}&publishedAfter={start_date}&publishedBefore={end_date}&key={key}'
    response = requests.get(url)  # Requête pour rechercher des vidéos
    data = response.json()

    videos = []
    for video in data['items']:
        video_info = {
            'video_id': video['id']['videoId'],
            'title': video['snippet']['title'],
            'description': video['snippet']['description'],
            'published_at': video['snippet']['publishedAt'],
            'channel_title': video['snippet']['channelTitle']
        }	
        videos.append(video_info)

    # Récupérer les pages suivantes si nécessaire
    next_page_token = data.get('nextPageToken')
    while next_page_token:
        url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&q={query}&type=video&maxResults={max_results}&pageToken={next_page_token}&publishedAfter={start_date}&publishedBefore={end_date}&key={key}'
        response = requests.get(url)
        data = response.json()
        for video in data['items']:
            video_info = {
                'video_id': video['id']['videoId'],
                'title': video['snippet']['title'],
                'description': video['snippet']['description'],
                'published_at': video['snippet']['publishedAt'],
                'channel_title': video['snippet']['channelTitle']
            }
            videos.append(video_info)
        next_page_token = data.get('nextPageToken')

    return videos

# Appel de la fonction pour récupérer les vidéos entre 2019 et 2023
query = "livre"
videos = get_videos(query, start_date="2019-01-01T00:00:00Z", end_date="2023-12-31T23:59:59Z", max_results=50, key)

# Convertir les données en DataFrame
df = pd.DataFrame(videos)

# Sauvegarder le DataFrame sous forme de fichier CSV
df.to_csv('youtube_livre_2019_2023.csv', index=False)

print("Le fichier CSV a été créé avec succès !")

Le fichier CSV a été créé avec succès !
